In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer.trainer_classifier import Trainer_classifier
from src.trainer.trainer_VAE import Trainer_VAE
from src.models.classifiers import *
from src.trainer.model_class import Model_class
from src.trainer.loss_class import Loss_class

from src.models.autoencoders import *
from src.models.joinedModel import *

from src.tools import upsampling

import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
HIDDEN_PARAM = 512
LATENT_REPR = 4
BATCH_SIZE = 1024

In [3]:
df = pd.read_csv('../data/df_to_enc.csv')

In [4]:
def prepare_data_for_test(X_data, y_data, test_ratio):
    # Prepare dataset for testing
    X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                     y_data,
                                                     shuffle = True,
                                                     stratify = y_data,
                                                     random_state = seed_value,
                                                     test_size = test_ratio)
    
    return X_train, X_test, y_train, y_test

In [5]:
def prepare_data_for_enc(X_data, y_data, autoenc_requared):

    # Check that amount rows for enc less than length of data
    if autoenc_requared >= len(X_data):
        raise ValueError("The number of rows for autoencoder more than amount of X_train data")
    
    autoenc_ratio = autoenc_requared/len(X_data)
    
    X_to_enc, X_to_clas,\
    y_to_enc, y_to_clas = train_test_split(X_data,
                                           y_data,
                                           shuffle = True,
                                           stratify = y_data,
                                           random_state = seed_value,
                                           train_size = autoenc_ratio)
    
    X_encoder_train, X_encoder_test = train_test_split(X_to_enc,
                                       shuffle = True,
                                       random_state = seed_value,
                                       train_size = 0.9)
    
    return X_encoder_train, X_encoder_test, X_to_clas, y_to_clas

In [6]:
def prepare_data_for_classif(X_data, y_data, classif_requared):

    # Prepare dataset for encoder
    if classif_requared >= len(X_data):
        raise ValueError("The number of rows for classifier more than amount of X_train data")
    classif_ratio = classif_requared / len(X_data)
    
    X_train, X_test,\
    y_train, y_test = train_test_split(X_data,
                                       y_data,
                                       shuffle = True,
                                       stratify = y_data,
                                       random_state = seed_value,
                                       train_size = classif_ratio)
    
    return X_train, y_train

In [7]:
def make_dataloader(*data, encoder_data = False):
    
    if len(data) > 1:
        data_list = [data[i] for i in range(len(data))]
        dataset = pd.concat(data_list, axis = 1)
    else:
        dataset = data[0]
    if encoder_data == False:
        dataset = TableDatasetDF(dataset)
        dataloader = DataLoader(
            dataset,
            batch_size=BATCH_SIZE, 
            shuffle=True,
            generator=generator
        )
    else:
        dataset = EncoderDataset(dataset)
        dataloader = DataLoader(
            dataset,
            batch_size=BATCH_SIZE, 
            shuffle=True,
            generator=generator
        )

    return dataloader

In [8]:
def prepare_data(X_data, y_data, test_ratio, autoenc_requared, classif_requared):
    X_train, X_test, y_train, y_test = prepare_data_for_test(X_data, y_data, test_ratio)
    X_encoder_train, X_encoder_test, X_to_clas, y_to_clas = prepare_data_for_enc(X_train, y_train, autoenc_requared)
    X_train_classif, y_train_classif = prepare_data_for_classif(X_to_clas, y_to_clas, classif_requared)

    X_train_classif, y_train_classif = upsampling(0.35, X_train_classif, y_train_classif)

    test_dl = make_dataloader(X_test, y_test)
    train_dl = make_dataloader(X_train_classif, y_train_classif)

    enc_train_dl = make_dataloader(X_encoder_train, encoder_data=True)
    enc_test_dl = make_dataloader(X_encoder_test, encoder_data=True)

    return train_dl, test_dl, enc_train_dl, enc_test_dl

In [9]:
def train_encoder(train_dl, test_ld, VAE_model = False):
        
        classifier = Baseline_classifier(train_dl.dataset.data.shape[1], HIDDEN_PARAM)

        if VAE_model == False:
            autoencoder = Autoencoder(classifier.classifier, 5)
            loss = Encoder_loss(nn.MSELoss())
        else:
            autoencoder = VAE(classifier.classifier, 5)
            loss = vae_loss(vae_loss_function)

        model_factory = partial(Model_class)
        optimizer_factory = partial(torch.optim.AdamW)
        scheduler_factory = partial(lr.ExponentialLR)

        model_params = dict(model=autoencoder,
                            device=device)

        optimizer_params = dict(weight_decay=1e-3, lr=1e-1)
        scheduler_params = dict(gamma=0.95)

        learning_params = dict(batch_size=BATCH_SIZE, num_epoch=200)

        if VAE_model == False:
            wandb_init_params = dict(
                name=f'Autoencoder_simple_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
                project="Internship_project",
                dir = '../logs/')
            
            trainer = Trainer_classifier(train_dl,
                          test_ld,
                          loss,
                          model_factory=model_factory,
                          optimizer_factory=optimizer_factory,
                          scheduler_factory=scheduler_factory,
                          model_params=model_params,
                          optimizer_params=optimizer_params,
                          scheduler_params=scheduler_params,
                          log=False,
                          wandb_init_params=wandb_init_params,
                          model_dir='../logs/nn_models/autoencoder/',
                          saving_model=False
                          )
        else:
            wandb_init_params = dict(
                name=f'VAE_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
                project="Internship_project",
                dir = '../logs/')
            
            trainer = Trainer_VAE(train_dl,
                          test_ld,
                          loss,
                          model_factory=model_factory,
                          optimizer_factory=optimizer_factory,
                          scheduler_factory=scheduler_factory,
                          model_params=model_params,
                          optimizer_params=optimizer_params,
                          scheduler_params=scheduler_params,
                          log=False,
                          wandb_init_params=wandb_init_params,
                          model_dir='../logs/nn_models/autoencoder/',
                          saving_model=False
                          )
        
        
        trainer.train_model(learning_params)
        wandb.finish()
        
        return trainer.model.model.encoder

In [10]:
def train_classifier(encoder, train_dl, test_dl, labels_amount, VAE_model = False):

    if VAE_model == False:
        classifier = Simple_classifier(LATENT_REPR)
        jm = JoinedModel(encoder, classifier)

        wandb_init_params = dict(
        name=f'JMenc_NumLab-{labels_amount}_LatDim-{LATENT_REPR}',
        project="Internship_project",
        dir = '../logs/')

    else:
        classifier = Simple_classifier(LATENT_REPR)
        jm = JoinedModel(encoder, classifier)
        wandb_init_params = dict(
        name=f'JMvae_NumLab-{labels_amount}_LatDim-{LATENT_REPR}',
        project="Internship_project",
        dir = '../logs/')

    loss = Loss_class(FocalLoss(gamma=2))
    model_factory = partial(Model_class)
    optimizer_factory = partial(torch.optim.AdamW)
    scheduler_factory = partial(lr.ExponentialLR)

    model_params = dict(model=jm,
                        device=device)

    optimizer_params = dict(weight_decay=1e-3, lr=1e-1)
    scheduler_params = dict(gamma=0.95)

    learning_params = dict(batch_size=BATCH_SIZE, num_epoch=150)

    trainer = Trainer_classifier(train_dl,
                        test_dl,
                        loss,
                        model_factory=model_factory,
                        optimizer_factory=optimizer_factory,
                        scheduler_factory=scheduler_factory,
                        model_params=model_params,
                        optimizer_params=optimizer_params,
                        scheduler_params=scheduler_params,
                        log=True,
                        wandb_init_params=wandb_init_params,
                        model_dir='../logs/nn_models/joined_models/',
                        saving_model=False
                        )
    
    trainer.train_model(learning_params)
    wandb.finish()

In [11]:
def train_cycle(df, list_amount = [50, 100, 500, 1000, 1990]):
    for labels_amount in list_amount:
        ############################################################
        # PREPARE DATA
        ############################################################a
        train_dl, test_dl, enc_train_dl, enc_test_dl = prepare_data(df.drop(columns = ['Machine failure']),
                                                                    df['Machine failure'], 0.2, 6000, labels_amount)
        
        
        ############################################################
        # Autoenc cycle
        ############################################################
        encoder_to_classif = train_encoder(enc_train_dl, enc_test_dl)
        train_classifier(encoder_to_classif, train_dl, test_dl,labels_amount)
        
        ############################################################
        # VAE cycle
        ############################################################
        encoder_to_classif = train_encoder(enc_train_dl, enc_test_dl, VAE_model = True)
        train_classifier(encoder_to_classif, train_dl, test_dl,labels_amount, VAE_model = True)

In [12]:
train_cycle(df)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,████▁▃█████▇▇▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
test_auc_score,▅▆▃▃▁▃██▆▇▇▇█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
test_f1_score,▁▁▁█▁▂▂▁▂▃▆▇▇▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▅▅▅▅▅▅▅
test_fpr,▁▁▁▁█▇▁▁▁▁▂▂▃▃▄▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
test_loss,▅▄▄▇█▇▂▂▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
test_tpr,▁▁▁▄▆▆▁▁▁▂▃▄▅▆▆██▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇
train_accuracy,▁▁▁▁▃▃▃▄▄▄▄▄▄▄▄█████████████████████████
train_auc_score,▂▁▆█▇▇██████████████████████████████████
train_f1_score,▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅█████████████████████████


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▅▄▆▄▆▃█▅█▄▄▅▆▄▅▄▇▄▄▄▅▄▃▅▅▂▆▅▃▇▆▃▅▂▄▄▄▇▁▃
test_auc_score,▁▂▂▂▁▄▅▆█▇▇▇█▇▇▇▇▆▆▅▆▅▅▆▆▆▆▅▆▆▅▅▅▅▅▅▅▅▅▅
test_f1_score,▆▄▆▅▆▄█▆█▄▅▆▇▅▆▅▇▄▄▅▆▅▃▆▆▃▆▆▃▇▆▄▆▂▅▄▅▇▁▄
test_fpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▄▂▂▂▂▂▂▁▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
test_tpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_auc_score,▄▂█▁▅▅▄▅▅▃▃▃▃▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
train_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▅▆▃▅▆▃▅▄▇▆▄▆▂▇▅▃▅▄▆▃▆▅▃▃▅▅▂▃▅▆▃▅█▆▅▆▁▅▅▆
test_auc_score,▂█▁▄▇▃▃▃▄▅▅▅▅▆▅▅▅▆▆▅▆▅▅▆▅▆▆▅▅▆▅▅▅▅▅▆▆▅▅▆
test_f1_score,▅▆▃▆▇▃▆▄▇▇▅▇▃▇▆▃▅▄▆▄▆▆▄▄▆▆▃▄▆▇▄▆█▇▅▆▁▆▅▇
test_fpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_tpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_auc_score,▁▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
train_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,█████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_auc_score,▅▄▆█▁▅▅▃▃▃▃▃▃▄▄▄▄▄▄▅▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▄▅▅
test_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████▇██▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇
test_fpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇█████████████████████
test_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_tpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇█▇▇██▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████
train_auc_score,▁█▇▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
train_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁███████████████████████████████████████
test_auc_score,▁▅█▆█▅▃▄▅▄▃▃▃▂▂▂▁▁▁▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
test_f1_score,▁█████████████▇█████████████████████████
test_fpr,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_tpr,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
train_accuracy,▁███████████████████████████████████████
train_auc_score,▁███████████████████████████████████████
train_f1_score,▁█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,█████████████████████████▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
test_auc_score,▁█▇▆▅▅▆▅▅▅▅▅▅▅▅▅▅▅▅▆▅▅▅▅▆▅▅▅▅▅▅▆▆▅▅▆▆▅▅▆
test_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇▇▇▇▇▇██████▇▇
test_fpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████▇██
test_loss,█▁▁▁▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▆▅▅▆▆▆▅▅▅▅▅▅▅▅
test_tpr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███▇▇▇██▇████▇█
train_accuracy,▂▂▂▁▂▃▃▂▃▂▁▂▂▃▃▂▃▂▁▁▁▂▂▁▁▇▄▃▃▃▆▆█▄▇▅█▇▅▇
train_auc_score,▁▆▇▇▇▇▇█▇▇▇▇▆█▇▇▇█▇▇▇▇▇█▇▇▆▇▇▇▇▇█▇▇▇██▇█
train_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▄▄▄▄▇▇█▄█▄██▇█


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,████████▅▆▅▆▆▆▅▆▆▆▁▅▂▄▆▅▅▆▅▅▆▅▄▅▅▅▄▅▆▄▅▃
test_auc_score,▁▁▁▁▁▁▂▆▇██▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇████▇█▇▇▇█▇▇
test_f1_score,▁▁▁▁▁▁▁▁▄▆▆██▇████▇▇▇▇█▇▇████▆▇▇▇█▇▇█▇▇▇
test_fpr,▁▁▁▁▁▁▁▁▄▄▅▄▄▄▅▄▄▄█▅▇▆▅▅▅▅▅▅▅▅▆▅▅▅▅▅▅▆▅▆
test_loss,█▅▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_tpr,▁▁▁▁▁▁▁▁▄▆▆██▇███▇█▇█▇█▇▇████▆█▇██▇▇██▇█
train_accuracy,▁▄▄▅▃▇▂▆▅▆▅▄▃▇▄█▅▆█▃▂▅▆▅▄▄▄█▅▅▄▅▆▄▅▇▆▄▃▄
train_auc_score,▁▃█▅▅▆▄█▇▄▄▇▄▄▇▅▅▅▆▅▅▆▄▄▄▄▆▇▆▆▃▅█▅▇▄▅▆▇▄
train_f1_score,▁▄▆▇▄█▃▄▇▇▄▄▇▇▇█▄▆█▅▃▆▆▆▄▅▄█▅▅▇▇▆▇▅█▆▇▄▆


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,██▅▂▂▁▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
test_auc_score,▁▇██▇███▇▇▇▇██▇▇▇▇▇▇██▇▇▇▇▇▇▇▇█▇█▇▇▇█▇█▇
test_f1_score,▇▅█▄▂▁▃▂▃▃▃▃▃▃▃▃▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
test_fpr,▁▁▅▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
test_loss,█▁▂▄▄▄▃▄▃▄▄▃▄▄▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
test_tpr,▁▁▇███▇█▇▇▇▇█▇▇█▇▇█████▇█▇█▇▇█▇▇▇▇▇▇█▇█▇
train_accuracy,▁▃▆█▅▅▆▆▇▆▆▆▆▇▇▇▆▆▅▅▆▆▆▆▇▇▅▆▆▇▅▆▅▅▇▄▇▅▅▆
train_auc_score,▁▅██▇▆▆▇▇▇█▇▇██▇▇▇▇▆▇▇████▇▇▇█▇▇▇██▇█▇▇█
train_f1_score,▅▁▇█▆▆▇▇█▇▆▇▇█▇▇▇▇▆▇▇▇▇▇█▇▆▇▇▇▆▇▇▆█▆█▆▆▇


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,███▁▅▅▆▆▆▆▇▇▆▂▃▄▅▄▅▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
test_auc_score,▂▂▁█████▇▇▇▇▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇
test_f1_score,▃▃▃▁▇▇▇█▇▆▇▇▆▁▂▅▅▄▅▅▆▆▆▅▅▅▅▆▅▅▅▅▅▅▅▅▅▅▅▅
test_fpr,▁▁▁█▄▄▄▃▄▃▂▂▃▇▆▅▄▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
test_loss,▁▁▁█▄▃▃▂▃▂▁▂▃▇▆▄▄▄▄▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
test_tpr,▁▁▁█▇▆▅▅▅▄▃▃▄▇▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
train_accuracy,▁▁▄▆▆▆▆▆▇▇▇▇▇█▇█▇███▇█▇▇▇▇▇█▇▇██▇▇███▇██
train_auc_score,▁▃▂▅▅▆▇▆▇▇▇▇████████▇▇▇▇▇███▇▇█▇█▇█▇█▇██
train_f1_score,▁▁▆▇▇▇▇▇███▇▇███████▇████▇██████████████
